<a href="https://colab.research.google.com/github/1o-o1/Smart-Contract/blob/main/smart_contract_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Research/Smart Contract
%ls

Mounted at /content/drive
/content/drive/MyDrive/Research/Smart Contract
 18-02-23.pptx             Dataset.zip
'=3.1'                     Docs/
 ANN0_acc.png              Ethereum_smart_contract_datast.zip
 ANN0_loss.png             filenames2.csv
 ANN0.png                  filenames3.csv
 ANN0result.csv            filenames.csv
 ANN0result.gsheet         files3.csv
 BRB.ipynb                 FirstRunMLresult.csv
'BRB with XAI.ipynb'       FirstRunMLresult.gsheet
 bytecode@                 FirstRun-Test-result.csv
 CNN0_acc.png              FirstRun-Train-result.csv
 CNN0_loss.png             FirstRun-Train-result.gsheet
 CNN0.png                  Generic_Select512.pickle
 CNN0result.csv            input2.csv
 CNN0result.gsheet         input3csv
 CNN1_acc.png              input.csv
 CNN1_loss.png             Label2.csv
 CNN1.png                  Label3.csv
 CNN1result.csv            Label.csv
'Copy of XProcessed.csv'  'lit (1).csv'
'Copy of yProcessed.csv'  'ML Run.ipynb'
 dataset/   

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import time
import os
import sklearn.metrics as m
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
! pip install py-solc-x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import solcx
ver=solcx.get_installable_solc_versions()

In [ ]:
for v in ver:
  solcx.install_solc(v)

In [ ]:
!pip install astropy>=3.1
!pip install sunpy
!pip install evm-cfg-builder
!pip install stellargraph

In [ ]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement stellargraph (from versions: none)
ERROR: No matching distribution found for stellargraph


In [ ]:
from evm_cfg_builder import CFG
from solcx import  compile_files
import solcx
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from stellargraph.data import BiasedRandomWalk
from keras.preprocessing.text import Tokenizer

In [ ]:
filename="/content/drive/MyDrive/Research/Smart Contract/Dataset 12K Eth 8 vol/block number dependency (BN)/0.sol"

In [ ]:
#@title
def PreProccess(filename):
      strt=0
      end=0
      walkss=[]
      with open(filename, 'r') as file:
        soldata = file.read()
      for i in range(0,len(soldata)):
        if soldata[i:i+2]=='0.':
          strt=i
        if strt!=0 and soldata[i]==';':
          end=i
          break
      #print(soldata[0:50])
      #if end-strt<5:
      #  verson="0.4.24"
      #else:
      verson=soldata[strt:end]
      #print("verson="+verson)
      try:
        binary= solcx.compile_files([filename], output_values=["bin-runtime"],solc_version=verson)

        positions=list(binary.keys())
        for pos in positions:
          binruntime=binary[pos]["bin-runtime"]
          #print("len of code "+str(len(binruntime)))
          if len(binruntime)>16:
            cfg = CFG(binruntime)
            src=[]
            dist=[]
            #print(cfg)
            for basic_block in cfg.basic_blocks:
              for outgoing in basic_block.all_outgoing_basic_blocks:
                x= basic_block
                x= str(x).replace('<cfg BasicBlock@','')
                x= x.replace('>','')
                src.append(x)
                y= outgoing
                y= str(y).replace('<cfg BasicBlock@','')
                y= y.replace('>','')
                dist.append(y)

            graph=pd.DataFrame(zip(src,dist),columns=['source','target'])
            graph = StellarGraph(edges=graph)


            biased_rw = BiasedRandomWalk(graph)


            walk_length = 10
            #DFS
            walks = biased_rw.run(nodes=list(graph.nodes()), length=walk_length, n=1, p=1.0, q=1.0)

            if len(walks)>0:
            #print(walks[0])
              walks = [str(node) for walk in walks for node in walk]
              walkss.append(walks)
            #tokenizer = Tokenizer(num_words=128, filters='', lower=False, split=' ')

            # Fit the tokenizer on the walks
            #tokenizer.fit_on_texts(walks)

            # Convert the walks to numerical format
            #sequences = tokenizer.texts_to_sequences(walks)

        return [node for walk in walkss for node in walk]
      except:
        return 0


# New Section

In [ ]:
rel_dirname="/content/drive/MyDrive/Research/Smart Contract/Dataset 12K Eth 8 vol"

for dirname in os.listdir(rel_dirname):
  cnt=0
  for filename in glob(os.path.join(rel_dirname, dirname+'/*.sol')):
    cnt=cnt+1
  print(dirname+" "+str(cnt))

ether strict equality (SE) 366
ground truth label.csv 0
README.md 0
block number dependency (BN) 406
dangerous delegatecall (DE) 97
ether frozen (EF) 97
integer overflow (OF) 590
reentrancy (RE) 1218
timestamp dependency (TP) 312
unchecked external call (UC) 1199
SpankChain_Payment.sol 0
Reentrancy.sol 0


In [ ]:
y=pd.read_csv("yProcessed.csv")
y.value_counts()

0
0    2802
1    2688
2    1680
3    1074
4     720
5     600
6     378
7     378
dtype: int64

In [ ]:
dataset=[]
label=[]
files=[]
filename="/content/drive/MyDrive/Research/Smart Contract/Dataset 12K Eth 8 vol/block number dependency (BN)/0.sol"
rel_dirname="/content/drive/MyDrive/Research/Smart Contract/Dataset 12K Eth 8 vol"
cnt=0
for dirname in os.listdir(rel_dirname):
    for filename in glob(os.path.join(rel_dirname, dirname+'/*.sol')):
      #print(filename)
      try:
        tmp=PreProccess(filename)
        #print(tmp)

        if tmp!=0:
          dataset.append(tmp)
          label.append(dirname)
          files.append(filename)
      except:
           cnt=cnt+1







<ipython-input-17-3f2280c29a27>:52: RuntimeWarning: No root node IDs given. An empty list will be returned as a result.
  walks = biased_rw.run(nodes=list(graph.nodes()), length=walk_length, n=1, p=1.0, q=1.0)


In [ ]:
print(cnt)

10


In [ ]:
len(files)

2578

In [ ]:
tokenizer = Tokenizer(num_words=1024, filters='', lower=False, split='')

        # Fit the tokenizer on the walks
tokenizer.fit_on_texts(dataset)

        # Convert the walks to numerical format
dataset = tokenizer.texts_to_sequences(dataset)

In [ ]:
import pickle

# saving
with open('tokenizer2.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
"""
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
"""

"\nwith open('tokenizer.pickle', 'rb') as handle:\n    tokenizer = pickle.load(handle)\n"

In [ ]:
len(dataset)

2552

In [ ]:
df=pd.DataFrame(dataset)

In [ ]:
df.shape

(2578, 4451)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4441,4442,4443,4444,4445,4446,4447,4448,4449,4450
0,10.0,11.0,11.0,12.0,13.0,608.0,13.0,13.0,8.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.0,13.0,14.0,13.0,14.0,13.0,12.0,12.0,12.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80.0,676.0,80.0,2.0,80.0,2.0,1.0,2.0,1.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33.0,35.0,57.0,57.0,57.0,8.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.0,7.0,6.0,7.0,8.0,8.0,7.0,12.0,13.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop the rows where all elements are missing

In [ ]:
df.dropna(how='all')

,0,1,2,3,4,5,6,7,8,9,...,4441,4442,4443,4444,4445,4446,4447,4448,4449,4450
0,10.0,11.0,11.0,12.0,13.0,608.0,13.0,13.0,8.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.0,13.0,14.0,13.0,14.0,13.0,12.0,12.0,12.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80.0,676.0,80.0,2.0,80.0,2.0,1.0,2.0,1.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33.0,35.0,57.0,57.0,57.0,8.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.0,7.0,6.0,7.0,8.0,8.0,7.0,12.0,13.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573,12.0,11.0,10.0,11.0,10.0,9.0,8.0,4.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2574,7.0,6.0,13.0,13.0,12.0,11.0,8.0,7.0,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2575,7.0,8.0,885.0,885.0,885.0,885.0,6.0,7.0,11.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2576,37.0,37.0,39.0,41.0,43.0,39.0,37.0,34.0,34.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range (500,600):
  print((df[i]==-1.0).sum())

In [ ]:
df=df.fillna(-1)

In [ ]:
df = df.replace("None", -1)

In [ ]:
df.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
251    float64
252    float64
253    float64
254    float64
255    float64
Length: 256, dtype: object

In [ ]:
df.to_csv('input3csv',index=False)

In [ ]:
pd.DataFrame.from_dict(label).to_csv('Label3.csv',index=False)

In [ ]:
pd.DataFrame.from_dict(files).to_csv('files3.csv',index=False)

In [ ]:
x=np.array(dataset)

<ipython-input-22-6149d69bee2a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x=np.array(dataset)


In [ ]:
xt=np.array(df)

In [ ]:
#xxt=np.concatenate((df,df,df,df,df),axis=0)

In [ ]:
xxt.shape

(15312, 4284)

In [ ]:
y=pd.DataFrame(label)
y.value_counts()

reentrancy (RE)                 556
unchecked external call (UC)    537
integer overflow (OF)           429
block number dependency (BN)    374
ether strict equality (SE)      315
timestamp dependency (TP)       221
dangerous delegatecall (DE)      73
ether frozen (EF)                73
dtype: int64

In [ ]:
y=y.replace("reentrancy (RE)", 0)
y=y.replace("unchecked external call (UC)", 1)
y=y.replace("integer overflow (OF)", 2)
y=y.replace("ether strict equality (SE)", 3)
y=y.replace("timestamp dependency (TP)", 4)
y=y.replace("block number dependency (BN)", 5)
y=y.replace("dangerous delegatecall (DE)", 6)
y=y.replace("ether frozen (EF)", 7)

In [ ]:
y.value_counts()

0    547
1    528
2    429
5    371
3    308
4    221
6     73
7     73
dtype: int64

In [ ]:
#yyt=np.concatenate((y,y,y,y,y,y),axis=0)

In [ ]:
#filesx=np.concatenate((files,files,files,files,files,files),axis=0)

In [ ]:
y.shape

(15312, 1)

In [ ]:
from sklearn.feature_selection import GenericUnivariateSelect, chi2, mutual_info_classif
transformer = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=256)
X_new = transformer.fit_transform(xxt, yyt)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
pd.DataFrame(xxt).to_csv('datasetx2.csv',index=False)
pd.DataFrame(yyt).to_csv('datasety2.csv',index=False)
pd.DataFrame(filesx).to_csv('filenames2.csv',index=False)

In [ ]:
df=pd.read_csv('input3csv')

In [ ]:
y=pd.read_csv('Label3.csv')
fall=pd.read_csv('files3.csv')

In [ ]:
y=y.replace("reentrancy (RE)", 0)
y=y.replace("unchecked external call (UC)", 1)
y=y.replace("integer overflow (OF)", 2)
y=y.replace("ether strict equality (SE)", 3)
y=y.replace("timestamp dependency (TP)", 4)
y=y.replace("block number dependency (BN)", 5)
y=y.replace("dangerous delegatecall (DE)", 6)
y=y.replace("ether frozen (EF)", 7)

In [ ]:
xxt.shape

(10312, 4451)

In [ ]:
yyt.shape

(10312, 1)

In [ ]:
files.shape

(10312, 1)

In [ ]:
pd.DataFrame(xxt).to_csv('datasetxraw.csv',index=False)
pd.DataFrame(yyt).to_csv('datasety3.csv',index=False)
pd.DataFrame(files).to_csv('filenames3.csv',index=False)